In [1]:
import asyncio
import json
import requests


from app.utility.helps import Bob

In [2]:
bob = Bob()
settings = bob.get_settings()
headers = bob.get_context()

In [3]:
api_url = 'https://api.powerbi.com/v1.0/myorg/gateways'
response = requests.get(url=api_url, headers=headers)
results = response.json()
print(response.status_code)
gateways = list()
for gateway in results['value']:
    gateways.append(gateway.get("id"))


200


In [4]:
r = list()
for id in gateways:
    response = requests.get(f'https://api.powerbi.com/v1.0/myorg/gateways/{id}/datasources', headers=headers)
    doc_results = response.json()
    r.append(doc_results['value'])   


In [5]:
gateways = list()

for i in range(0, len(r)):
    if isinstance(r[i], list):
        for j in range(0, len(r[i])):
            gateways.append(r[i][j])
    else:
        gateways.append(r[i])

In [9]:
gateways

[{'id': '4af4f08c-2107-479b-92cf-2a0dfca9b866',
  'gatewayId': '04d27518-5e41-47e0-a1f9-3fbe66361826',
  'datasourceType': 'File',
  'connectionDetails': '{"path":"C:\\\\Projects\\\\Ecolab\\\\Geospatial\\\\SF Data US Only.xlsx"}',
  'credentialType': 'Windows',
  'credentialDetails': {'privacyLevel': 'None',
   'useEndUserOAuth2Credentials': False},
  'datasourceName': 'C:\\Projects\\Ecolab\\Geospatial\\SF Data US Only.xlsx'},
 {'id': 'a62a82bb-f449-4493-92ec-200054295550',
  'gatewayId': '04d27518-5e41-47e0-a1f9-3fbe66361826',
  'datasourceType': 'Extension',
  'connectionDetails': '{"extensionDataSourceKind":"PowerPlatformDataflows","extensionDataSourcePath":"PowerPlatformDataflows"}',
  'credentialType': 'OAuth2',
  'credentialDetails': {'privacyLevel': 'Public',
   'useEndUserOAuth2Credentials': False},
  'datasourceName': 'mydatagateway'},
 {'id': '4a4e8b34-a5d9-454e-9d86-0b987996a80f',
  'gatewayId': 'b910099b-c9d9-4f90-b4cb-6a1bf61244cd',
  'datasourceType': 'Extension',
  'conn

In [6]:
users = list()

for i in range(0,len(gateways)):
    api_users = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}/users'
    response = requests.get(api_users, headers=headers)
    results = response.json()
    results['value'][0]['datasourceId'] = gateways[i]["id"]
    results['value'][0]['gatewayId'] = gateways[i]["gatewayId"]
    users.append(results['value'][0])
    

## Datasource Connectivity  

This only reports back connectivity to the datasource as an error. Also, you need to use the response.text as the JSON is not populated. A successful connection will report back as HTTP 200 while a failed connection will result in HTTP 400

In [7]:
## This will be used for a template dictionary for a successful connection to the datasource
status_base = {   
    "datasourceId":"",
    "gatewayId":"",
    "error":
    {
        "code":"success",
        "pbi.error":{ 
            "code":"0",
            "parameters":{},
            "details":[],
            "exceptionCulprit":0
        }
    }
}


In [8]:
status  = list()
for i in range(0,len(gateways)):
    api_status = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}/status'
    
    response = requests.get(api_status, headers=headers)
    if response.ok:
        # response.json() is actually empty when it is a success
        
        status_base['datasourceId'] = gateways[i]["id"]
        status_base['gatewayId'] = gateways[i]["gatewayId"]
        status_base.get("error").get("pbi.error").get("details").append({"message":"success","detail":response.status_code})    
        status.append(status_base)
    else:
        results = json.loads(response.text)
        results["datasourceId"] = gateways[i]["id"]
        results["gatewayId"] = gateways[i]["gatewayId"]
        results.get("error").get("pbi.error").get("details").append({"message":"cannot connect","detail":response.status_code})    
        status.append(results)


In [11]:
datasources = list()

for i in range(0,len(gateways)):
    api_datasource = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}'
    response = requests.get(api_datasource, headers=headers)
    results = response.json()
    results.pop('@odata.context')
    datasources.append(results)


In [12]:
datasources

[{'id': '4af4f08c-2107-479b-92cf-2a0dfca9b866',
  'gatewayId': '04d27518-5e41-47e0-a1f9-3fbe66361826',
  'datasourceType': 'File',
  'connectionDetails': '{"path":"C:\\\\Projects\\\\Ecolab\\\\Geospatial\\\\SF Data US Only.xlsx"}',
  'credentialType': 'Windows',
  'credentialDetails': {'privacyLevel': 'None',
   'useEndUserOAuth2Credentials': False},
  'datasourceName': 'C:\\Projects\\Ecolab\\Geospatial\\SF Data US Only.xlsx'},
 {'id': 'a62a82bb-f449-4493-92ec-200054295550',
  'gatewayId': '04d27518-5e41-47e0-a1f9-3fbe66361826',
  'datasourceType': 'Extension',
  'connectionDetails': '{"extensionDataSourceKind":"PowerPlatformDataflows","extensionDataSourcePath":"PowerPlatformDataflows"}',
  'credentialType': 'OAuth2',
  'credentialDetails': {'privacyLevel': 'Public',
   'useEndUserOAuth2Credentials': False},
  'datasourceName': 'mydatagateway'},
 {'id': '4a4e8b34-a5d9-454e-9d86-0b987996a80f',
  'gatewayId': 'b910099b-c9d9-4f90-b4cb-6a1bf61244cd',
  'datasourceType': 'Extension',
  'conn

In [ ]:
import smtplib
from email.mime.text import MIMEText

sender_email = 'brcampb@microsoft.com'
receiver_email = 'brandonh.campbell@gmail.com'
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = receiver_email

# Set up the SMTP server (e.g., Gmail)
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
    server.login(sender_email, 'your_password')
    server.sendmail(sender_email, [receiver_email], msg.as_string())



sender_email = 'your_email@example.com'
receiver_email = 'recipient_email@example.com'
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = receiver_email

# Set up the SMTP server (e.g., Outlook)
with smtplib.SMTP('smtp.office365.com', 587) as server:
    server.starttls()
    server.login(sender_email, 'your_password')
    server.sendmail(sender_email, [receiver_email], msg.as_string())


In [ ]:
import smtplib
from email.mime.text import MIMEText

# Set up the SMTP server
smtp_server = 'smtp.gmail.com'
smtp_port = 587
sender_email = 'your_email@example.com'
password = 'your_password'

# Create a message
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'
msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = 'recipient_email@example.com'

# Send the email
with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()
    server.login(sender_email, password)
    server.sendmail(sender_email, [msg['To']], msg.as_string())
